In [1]:
import torch
import torchvision

In [2]:
dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=torchvision.transforms.ToTensor())


C:\Users\ashua\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
train_ds, val_ds = torch.utils.data.random_split(dataset, [50000, 10000])

print(len(train_ds), len(val_ds))

50000 10000


In [4]:
from torch.utils.data.dataloader import DataLoader

batch_size = 100

train_dl = DataLoader(train_ds, batch_size)
val_dl = DataLoader(val_ds, batch_size)

In [5]:
input_size = 28*28
num_classes = 10

class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)
        
    def forward(self, image):
        image = image.reshape(-1,784)
        return self.linear(image)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = torch.nn.functional.cross_entropy(out, labels) 
        return loss

    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = torch.nn.functional.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'loss': loss, 'accuracy': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accuracy = [x['accuracy'] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {'loss': epoch_loss.item(), 'accuracy': epoch_accuracy.item()}
        
    def epoch_end(self, epoch, result):
        print("Epoch [{}], loss: {:.4f}, accuracy: {:.4f}".format(epoch, result['loss'], result['accuracy']))
        
model = MnistModel()    

print(model.linear.weight.shape,model.linear.bias.shape)
list(model.parameters())

torch.Size([10, 784]) torch.Size([10])


[Parameter containing:
 tensor([[ 1.8717e-02,  2.5655e-02, -1.9281e-02,  ...,  2.4196e-02,
           1.9529e-03,  2.8096e-02],
         [ 4.6633e-05, -2.3835e-02, -1.8433e-02,  ...,  1.5085e-03,
          -2.3998e-02, -1.8777e-02],
         [ 3.3685e-02,  3.4502e-02, -4.9997e-03,  ..., -2.4566e-02,
          -2.5581e-03, -1.0365e-02],
         ...,
         [-2.7373e-02, -1.1421e-04, -5.2387e-03,  ...,  3.1517e-03,
          -2.6222e-02, -2.6995e-02],
         [-1.0259e-02,  2.9644e-02, -2.3885e-02,  ...,  3.4910e-02,
           5.7488e-03, -4.7861e-03],
         [-1.1815e-02, -1.0367e-03,  9.9360e-03,  ...,  3.2132e-02,
           2.4086e-02, -2.8986e-02]], requires_grad=True),
 Parameter containing:
 tensor([-0.0206,  0.0100,  0.0062,  0.0055, -0.0080,  0.0185,  0.0026,  0.0139,
          0.0039,  0.0258], requires_grad=True)]

In [6]:
for images, label in train_dl:
    print(images.shape)
    outputs = model(images)
    break

torch.Size([100, 1, 28, 28])


In [7]:
def accuracy(outputs, label):
    preds = torch.max(outputs, dim=1)

    return (sum(preds[1] == label)/ len(label))

print(accuracy(outputs,label))

tensor(0.2000)


In [8]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] 
    
    for epoch in range(epochs):
        
        for batch in train_loader:
            loss = model.training_step(batch)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [9]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [10]:
result0 = evaluate(model, val_dl)
print(result0)

In [ ]:
history1 = fit(3, 0.001, model, train_dl, val_dl)

In [ ]:
from PIL import Image
test_dataset = torchvision.datasets.MNIST(root="./data", train=False)

x=9


img, label = test_dataset[x]
img.resize((280,280)).show()

test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor())

def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    _, preds = torch.max(yb, dim=1)
    return preds[0].item()

img, label = test_dataset[x]
predict_image(img, model)


1